In [1]:
import quick_folium as qf
import folium
from folium import plugins
from folium.plugins import Draw, MiniMap
import geopandas as gpd, requests
from io import BytesIO

#
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [2]:
# Import data from Github
dist = gpd.read_parquet(BytesIO(requests.get('https://raw.githubusercontent.com/routineactivity/folium_crime_maps/main/sample_data/cp_districts.parquet').content)).to_crs('EPSG:4326')
nbhd = gpd.read_file(BytesIO(requests.get('https://raw.githubusercontent.com/routineactivity/folium_crime_maps/main/sample_data/cp_neighbourhoods.gpkg').content)).to_crs('EPSG:4326')
ward = gpd.read_parquet(BytesIO(requests.get('https://raw.githubusercontent.com/routineactivity/folium_crime_maps/main/sample_data/cp_wards2021.parquet').content)).to_crs('EPSG:4326')
pnts = gpd.read_parquet(BytesIO(requests.get('https://raw.githubusercontent.com/routineactivity/folium_crime_maps/main/sample_data/data_24mrobbery_clusters.parquet').content)).to_crs('EPSG:4326')
clst = gpd.read_parquet(BytesIO(requests.get('https://raw.githubusercontent.com/routineactivity/folium_crime_maps/main/sample_data/data_robbery_clusters.parquet').content)).to_crs('EPSG:4326')

In [3]:
# View quick folium functions - map_location
help(qf.map_location)

Help on function map_location in module quick_folium:

map_location(gdf)
    Creates centroid for the study area from a geo data frame to use as Folium map location 
    
    Parameters:
    - a geodataframe: a geographic polygon layer.



In [4]:
# Use quick folium map_location
m = folium.Map(location=qf.map_location(gdf=dist), zoom_start=12, tiles=None)
folium.TileLayer('CartoDB positron').add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m

In [5]:
# View quick folium functions - map_boundary_tooltip
help(qf.map_boundary_tooltip)

Help on function map_boundary_tooltip in module quick_folium:

map_boundary_tooltip(map_obj, geo_data, fg_name='Boundary_with_tooltip', show=False, tooltip_fields=None, popup_fields=None, fo=0, fc=None, lw=2, lc='black', name='Boundary')
    Adds a boundary layer with tooltips and popups to a Folium map.
    
    Parameters:
    - map_obj: a folium map obj (str)
    - geo_data: a geodataframe containing polygons
    - fg_name: a feature group name (str)
    - show: show the layer when loaded (bool)
    - tooltip_fields: geodataframe column header field names (str)
    - popup_fields: geodataframe column header field names (str)
    - fo: fill_opacity (dec)
    - fc: fill_color (str)
    - lw: line_width (int)
    - lc: line_color (str)
    - name: new layer name (str)



In [6]:
# Use quick folium map_boundary_tooltip
m = folium.Map(location=qf.map_location(gdf=dist), zoom_start=12, tiles=None)

qf.map_boundary_tooltip(
    map_obj=m, 
    geo_data=nbhd, 
    fg_name='Cleveland Police Neighbourhoods', 
    show=True, 
    tooltip_fields=['police_area'], 
    popup_fields=['police_area'], 
    fo=0, 
    fc=None, 
    lw=2, 
    lc='black', 
    name='Cleveland Police Neighbourhoods')

folium.TileLayer('CartoDB positron').add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m

In [7]:
# Create multi-layer view - points and graduated points

# Create map using qf.map_location and dist 
m = folium.Map(location=qf.map_location(gdf=dist), zoom_start=12, tiles=None)

# Add map_boundary_tooltip with dist geodataframe
qf.map_boundary(
    geo_data=nbhd, 
    fg_name='Cleveland Police Neighbourhoods', 
    map_obj=m, 
    lw=3)

# Add map_point_label with pnts geodataframe
qf.map_point_label(
    map_obj=m, 
    geo_data=pnts, 
    name='Robbery Offences in Clusters', 
    r=6, 
    fc='black', 
    fo=1, 
    ec='black', 
    lw=1, 
    fields=['month', 'location', 'lad'], 
    show=False)

# Add map_graduated_circles with pnts geodataframe
qf.map_graduated_circles(
    map_obj=m,
    gdf_points=pnts,
    radius_scale=15,
    color='#17cbef',
    show=True,
    fill=True,
    fill_color='#17cbef',
    opacity=0.6,
    stroke=True,
    weight=1.0,
    fg_name='Robbery Graduated Circles')

# Add CartoPositron map tile layer and folium layer control
folium.TileLayer('CartoDB positron').add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

# View result
m

In [8]:
# Create multi-layer view - hotspot clusters and points

# Create map using qf.map_location and dist 
m = folium.Map(location=qf.map_location(gdf=dist), zoom_start=12, tiles=None)

# Add map_boundary_tooltip with dist geodataframe
qf.map_boundary(
    geo_data=nbhd, 
    fg_name='Cleveland Police Neighbourhoods', 
    map_obj=m, 
    lw=3)

# Add map_point_label with pnts geodataframe
qf.map_point_label(
    map_obj=m, 
    geo_data=pnts, 
    name='Robbery Offences in Clusters', 
    r=6, 
    fc='black', 
    fo=1, 
    ec='black', 
    lw=1, 
    fields=['month', 'location', 'lad'], 
    show=False)

# Add map_poly_label with clst geodataframe
qf.map_poly_label(
    map_obj=m, 
    geo_data=clst, 
    fg_name='Robbery Clusters', 
    lc='black', 
    lw=1.5, 
    tooltip_fields=['id', 'NUMPOINTS'], 
    tooltip_aliases=['AreaID: ', 'RobberyCount: '])

# Add CartoPositron map tile layer and folium layer control
folium.TileLayer('CartoDB positron').add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

# View result
m

In [9]:
# Final map - hotspot clusters, points and heatmap

# Create map using qf.map_location and dist 
m = folium.Map(location=qf.map_location(gdf=dist), zoom_start=12, tiles=None)

# Add map_boundary_tooltip with dist geodataframe
qf.map_boundary(
    geo_data=nbhd, 
    fg_name='Cleveland Police Neighbourhoods', 
    map_obj=m, 
    lw=3)

# Add map_point_label with pnts geodataframe
qf.map_point_label(
    map_obj=m, 
    geo_data=pnts, 
    name='Robbery Offences in Clusters', 
    r=6, 
    fc='black', 
    fo=1, 
    ec='black', 
    lw=1, 
    fields=['month', 'location', 'lad'], 
    show=True)

# Add map_poly_label with clst geodataframe
qf.map_poly_label(
    map_obj=m, 
    geo_data=clst, 
    fg_name='Robbery Clusters', 
    lc='black', 
    lw=1.5, 
    tooltip_fields=['id', 'NUMPOINTS'], 
    tooltip_aliases=['AreaID: ', 'RobberyCount: '])

# Add heatmap_from_points with pnts geodataframe
qf.heatmap_from_points(
    map_obj=m, 
    geo_data=pnts, 
    fg_name='Heatmap', 
    r=15, 
    b=10)

# Add CartoPositron map tile layer and folium layer control
folium.TileLayer('CartoDB positron').add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

# Add in a minimap
minimap = plugins.MiniMap()
m.add_child(minimap)

# Add in draw tools
Draw(export=False).add_to(m)

# View result
m

In [10]:
# Export
m.save("cp-robbery.html")